# Sensor Based Irrigation with ESP32

date 

Adar Sulema, Gabriel Bar Sella

The Faculty of Agriculture, The Hebrew University of Jerusalem (ISRAEL)

***

## Introduction

In this project we used data collected by PM-WCS-3 Soil sensor and HX711 load cell using ESP32 in order to find change in soil moisture and weight to create a sensor based irrigation system.  

Automated Irrigation is very common in these days with everyone from hobby gardeners to commercial growers using some form of computer controlled irrigation. Most common systems are open loop systems where irrigation is applied at fixed time intervals. 